In [58]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
sys.path.append('../scripts')
from sql_engine import connect
import batch_failure as bf
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
vendor_mapping = pd.read_excel(r'C:\Medline\CPM\data\vendor_mapping\Vendor _mapping 2023_v1.xlsx')
vendor_mapping_dict = dict(zip(vendor_mapping['Vendor Number'],vendor_mapping['Supervisor']))
    

In [60]:
fn_engine = connect('fn_mysql')
sql_query = r'''
                select
                ID,
                `PO Number`,
                `Lot Number`,
                `Vendor Code`,
                Vendor,
                Division,
                `Inspection Date`,
                Inspector,
                `Item Number`,
                Results,
                `Reject Code`,
                `Reject Description`,
                `Comments`,
                `Path`
            from
                inspection_data_all
            where 1 = 1
            and `Inspection Date` >= '2023-07-01'
            and Results = 'R'
            and `Reject Description` like '%%100\%%%%'
            order by `Inspection Date`   
            '''

In [61]:
df_add = pd.read_sql(sql_query,fn_engine)
df_old = pd.read_excel('../data_input/Rejection_data.xlsx')

In [62]:
df_QIM_duplicate = (
    df_add
    .query('Path.str.startswith("QIM",na=False)')
    .sort_values(['ID','Inspection Date'],ascending=[False,False])
    .drop_duplicates(subset=['ID'],keep='last')
    )

In [63]:
df_offline_duplicate = (
    df_add
    .query('~Path.str.startswith("QIM",na=False)')
    .sort_values(['ID','Inspection Date'],ascending=[False,False])
    .drop_duplicates(subset=['Lot Number','Vendor','Item Number','Inspector','Inspection Date'],keep='last')
    )

In [64]:
df_duplicate = pd.concat([df_QIM_duplicate,df_offline_duplicate],ignore_index=True)

In [65]:
df_duplicate = (
    df_duplicate
    .rename(columns={'Comments':'Disposition'})
    .assign(Supervisor = np.nan)
    .loc[:,['ID','Inspection Date','Supervisor','Inspector','Vendor Code','Vendor','Division','Item Number','PO Number','Lot Number','Reject Code','Reject Description','Disposition']]
)

In [66]:
(
    df_old
    .loc[:,['ID','Inspection Date','Supervisor','Inspector','Vendor Code','Vendor','Division','Item Number','PO Number','Lot Number','Reject Code','Reject Description','Disposition']]
    .query('`Reject Description`.str.contains("100%",na=False,case=False)')
    .pipe(lambda d : pd.concat([d,df_duplicate],ignore_index=True))
    .assign(Division = lambda d : d.Division.astype(str))
    .assign(Division = lambda d : d.Division.apply(bf.extract_division),
            Comments = lambda d : d['Disposition'].apply(bf.generate_remark),
            Supervisor = lambda d : d.apply(lambda s : vendor_mapping_dict.get(int(s['Vendor Code'])) if pd.isna(s['Supervisor']) else s['Supervisor'], axis = 1),
            Market = lambda d : d.Division.apply(bf.get_market)
            )
    
    .to_excel('../data_output/out.xlsx',index= False)
    
)